In [61]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import re 

# Example 4. PMF (Passenger Mix Flow) model

In [62]:
# Define the inputs

# L: set of flights
flights = pd.read_excel('Example_Data_PMF.xlsx', sheet_name='Flight').set_index('Flight Number')

# P: set of passenger itineraries
paths = pd.read_excel('Example_Data_PMF.xlsx', sheet_name='Itinerary').set_index('Itin No.')
paths['Leg 2'] = paths['Leg 2'].astype('Int64')

# P_p: set of passenger itineraries with recapture rate from itinerary p
P_p = pd.read_excel('Example_Data_PMF.xlsx', sheet_name='Recapture Rate').set_index(['p','r'])

# make a dictionary with the itinerary as the key and the rest as a sub-dictionary
paths = paths.to_dict(orient='index')
flights = flights.to_dict(orient='index')
P_p = P_p.to_dict(orient='index')

path_list = list(paths.keys())
flight_list = list(flights.keys())

# For all paths if 'Leg 1' and 'Leg 2' are numbers then create a list with both legs else, drop the keys from the list, and create a new key called 'Legs'
# else just change the name of the key 'Leg 1' to 'Legs'
for key in path_list:
    legs = []
    if type(paths[key]['Leg 1']) == int and type(paths[key]['Leg 2']) == int:
        legs.append(paths[key]['Leg 1'])
        legs.append(paths[key]['Leg 2'])
        paths[key]['Legs'] = legs
    elif type(paths[key]['Leg 1']) == int:
        legs.append(paths[key]['Leg 1'])
        paths[key]['Legs'] = legs
    del paths[key]['Leg 1']
    del paths[key]['Leg 2']

# Define path 999 with a fare of 0 and a demand of 0 
paths[999] = {'Legs': [], 'Demand': 0, 'Fare': 0}

In [63]:
# s_ip: binary variable indicating whether flight i is in itinerary p
s_ip = {}
for i in flight_list:
    for p in paths:
        s_ip[i,999] = 0
        if i in paths[p]['Legs']:
            s_ip[i,p] = 1
        else:
            s_ip[i,p] = 0

# Q_i: unconstrained demand for flight i = sum s_ip * demand of itinerary p for p in P
Q_i = {}
for i in flight_list:
    Q_i[i] = 0
    for p in paths:
        Q_i[i] += s_ip[i,p] * paths[p]['Demand']

# ds_i demand spill for flight i = Q_i - capacity of flight i
ds_i = {}
for i in flight_list:
    ds_i[i] = Q_i[i] - flights[i]['Capacity']

In [64]:
# Add entries to P_p for path 0 with a recapture rate of 1
for p in paths:
    P_p[p,999] = {'Recapture rate': 1}
    P_p[999,p] = {'Recapture rate': 0}

path_list = list(paths.keys())
flight_list = list(flights.keys())

In [65]:
initial = [999]

In [66]:
# Implementation using column generation algorithm (CGA)

# Define the restricted master problem (RMP)
# All the spillage is reallocated to path 999

# Define the model
m = gp.Model('PMF')

# Define the decision variables
# t_pr: number of passengers that would like to fly on itinerary p and are reallocated to itinerary r
t = {}
for p in path_list:
    for r in initial:
        t[p,r] = m.addVar(name='t_'+str(p)+'_'+str(r), lb=0)

In [67]:
# Define the objective function
# MINIMIZE (double sum of fare_p - bpr * fare_r) * t_pr
of = gp.quicksum((paths[p]['Fare'] - P_p[(p,r)]['Recapture rate'] * paths[r]['Fare']) * t[p,r] for r in initial for p in path_list)
m.setObjective(of, GRB.MINIMIZE)

# Define the constraints
# Constraint 1: sum sum s_ip * t_pr - sum sum s_ip * brp * t_rp >= ds_i for all i but for r = 0 
m.addConstrs((gp.quicksum(s_ip[i,p] * t[p,r] for p in path_list for r in initial) - 
              gp.quicksum(s_ip[i,p] * P_p[(r,p)]['Recapture rate'] * t[p,r] for p in path_list for r in initial) >= 
              ds_i[i] for i in flight_list), name='π')

# Constraint 2: sum t_pr <= Dp for all p
for p in path_list:
    m.addConstr((gp.quicksum(t[p,r] for r in initial) <= paths[p]['Demand']), name='σ[' + str(p) + ']')

# Update the model
m.update()

# Solve the model
m.optimize()


Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 26 rows, 16 columns and 36 nonzeros
Model fingerprint: 0xf2619915
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+01, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+00, 2e+02]
Presolve removed 23 rows and 11 columns
Presolve time: 0.01s
Presolved: 3 rows, 5 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.8055000e+04   1.162500e+01   0.000000e+00      0s
       2    4.6580000e+04   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.02 seconds (0.00 work units)
Optimal objective  4.658000000e+04


In [68]:
# Print the optimal objective value and the decision variables t_pr and the dual variables
print('Optimal objective value: %g' % m.objVal)
print('Optimal solution:')
for v in m.getVars():
    if v.x > 0:
        print('%s = %g' % (v.varName, v.x))

print('Dual variables:')
for c in m.getConstrs():
    if c.Pi != 0:
        print('%s = %g' % (c.ConstrName, c.Pi))

# Save dual variables in a dictionary
pi_dual = {}
for c in m.getConstrs():
    if c.constrName[0] == 'π':
        # get only the flight number from the constraint name
        flight_num_pi = int(re.findall(r'\d+', c.ConstrName)[0])    
        pi_dual[flight_num_pi] = c.Pi

sigma_dual = {}
for c in m.getConstrs():
    if c.constrName[0] == 'σ':
        #print(c.constrName)
        path_num_sigma = int(re.findall(r'\d+', c.ConstrName)[0])    
        sigma_dual[path_num_sigma] = c.Pi
        

Optimal objective value: 46580
Optimal solution:
t_2_999 = 12
t_5_999 = 50
t_6_999 = 12
t_7_999 = 62
t_8_999 = 70
t_9_999 = 32
t_10_999 = 87
t_12_999 = 72
t_13_999 = 30
Dual variables:
π[102] = 100
π[104] = 170
π[202] = 80
π[302] = 140
π[203] = 180
π[101] = 120
σ[5] = -90
σ[8] = -250
σ[13] = -100


In [69]:
def PMF_n_iters(pi, sigma, n_iters, paths, path_list, flight_list, P_p, initial, current_pairs = [], n=0):
    n += 1
    print('Iteration number: ', n)
    if n > n_iters:
        print('Max number of iterations reached')
        return ":("

    tpr_prime = {}
    # tpr = (fare_p - sum (π_i) for i being each flight in path p) - bpr * (fare_r - sum (π_j) for j being each flight in path p)) - σ_p
    for p,r in P_p.keys():
        t_prime_pr = ((paths[p]['Fare'] - sum(pi[i] for i in paths[p]['Legs'])) -
                        (P_p[(p,r)]['Recapture rate']) *
                        (paths[r]['Fare'] - sum(pi[j] for j in paths[r]['Legs'])) -
                        (sigma[p]))
        if t_prime_pr < 0:
            tpr_prime[p,r] = t_prime_pr
            print(str(p)+'->'+str(r)+': ', t_prime_pr)
    #print('tpr_prime: ', tpr_prime)
    new_pairs = list(tpr_prime.keys())
    current_pairs.extend(new_pairs)

    if len(new_pairs) == 0:
        print('No new pairs, optimal solution found in previous iteration')
    
    if len(new_pairs) > 0:
        print('New pairs: ', new_pairs)
        m_n = gp.Model(str(n)+'th PMF')
        # Define the decision variables
        # t_pr: number of passengers that would like to fly on itinerary p and are reallocated to itinerary r
        t = {}
        for p in path_list:
            for r in initial:
                t[p,r] = m_n.addVar(name='t_'+str(p)+'_'+str(r),vtype = GRB.CONTINUOUS)

        # Add the new columns to the RMP
        for p,r in current_pairs:
            t[p,r] = m_n.addVar(name='t_' + str(p) +'_'+ str(r), vtype = GRB.CONTINUOUS)
        m_n.update()

        # Update the objective function
        of  = gp.quicksum((paths[p]['Fare'] - P_p[(p,r)]['Recapture rate'] * paths[r]['Fare']) * t[p,r] for r in initial for p in path_list)
        of += gp.quicksum((paths[p]['Fare'] - P_p[(p,r)]['Recapture rate'] * paths[r]['Fare']) * t[p,r] for p,r in current_pairs)
        m_n.setObjective(of, GRB.MINIMIZE)

        # Update the constraints
        m_n.addConstrs((gp.quicksum(s_ip[i, p] * t[p,r] for p in path_list for r in initial) +
                    gp.quicksum(s_ip[i, p] * t[p,r] for p, r in current_pairs) -
                    gp.quicksum(s_ip[i, r] * P_p[(p, r)]['Recapture rate'] * t[p,r] for p,r in current_pairs) >=
                    ds_i[i] for i in flight_list), name='π')

        # Constraint 2: sum t_pr <= Dp for all p
        m_n.addConstrs((t[p,r] <= paths[p]['Demand'] for p in path_list for r in initial), name='σ')
        # Constraint 3: sum t_pr >= 0 for new pairs
        m_n.addConstrs((t[p,r] <= paths[p]['Demand'] for p, r in current_pairs), name='σ')

        # Constraint 4: sum t_pr >= 0 for all p
        m_n.addConstrs((t[p,r] >= 0 for p in path_list for r in initial), name='c3')
        # Constraint 5: sum t_pr >= 0 for new pairs
        m_n.addConstrs((t[p,r] >= 0 for p, r in current_pairs), name='c3')

        # Update the model
        m_n.update()

        # Solve the model but dont show the output
        m_n.Params.OutputFlag = 0
        m_n.optimize()

        # Print the optimal objective value and the decision variables t_pr and the dual variables
        print('Optimal objective value: %g' % m_n.objVal)
        print('\nOptimal solution:')
        for v in m_n.getVars():
            if v.x > 0:
                print('%s = %g' % (v.varName, v.x))

        # print('\nDual variables:')
        # for c in m_n.getConstrs():
        #     if c.Pi != 0:
        #         print('%s = %g' % (c.ConstrName, c.Pi))

        # Save dual variables in a dictionary
        pi_new = {}
        for c in m_n.getConstrs():
            if c.constrName[0] == 'π':
                # get only the flight number from the constraint name
                flight_num_pi = int(re.findall(r'\d+', c.ConstrName)[0])    
                pi_new[flight_num_pi] = c.Pi

        sigma_new = {}
        for c in m_n.getConstrs():
            if c.constrName[0] == 'σ':
                path_num_sigma = int(re.findall(r'\d+', c.ConstrName)[0])    
                sigma_new[path_num_sigma] = c.Pi
        
        print('End of iteration number: ', n, '\n')
        PMF_n_iters(pi = pi_new,
                    sigma = sigma_new, 
                    n_iters = n_iters, 
                    paths = paths, 
                    path_list = path_list, 
                    flight_list = flight_list, 
                    P_p = P_p, 
                    initial = initial, 
                    current_pairs = current_pairs, 
                    n=n)

In [70]:
PMF_n_iters(pi_dual, sigma_dual, 50, paths=paths, path_list=path_list, flight_list=flight_list, P_p=P_p, initial = initial, current_pairs = [], n=0)


Iteration number:  1
2->1:  -5.0
3->4:  -14.0
12->11:  -15.0
New pairs:  [(2, 1), (3, 4), (12, 11)]
Optimal objective value: 45006

Optimal solution:
t_5_999 = 50
t_6_999 = 12
t_7_999 = 63.2
t_8_999 = 70
t_9_999 = 32
t_10_999 = 56
t_13_999 = 30
t_2_1 = 12
t_3_4 = 31
t_12_11 = 72
End of iteration number:  1 

Iteration number:  2
No new pairs, optimal solution found in previous iteration


# Testing Code

In [71]:
# Create the pricing problem (PP)

tpr_prime = {}
# tpr = (fare_p - sum (π_i) for i being each flight in path p) - bpr * (fare_r - sum (π_j) for j being each flight in path p)) - σ_p
for p,r in P_p.keys():
    t_prime_pr = ((paths[p]['Fare'] - sum(pi_dual[i] for i in paths[p]['Legs'])) -
                      (P_p[(p,r)]['Recapture rate']) *
                      (paths[r]['Fare'] - sum(pi_dual[j] for j in paths[r]['Legs'])) -
                      (sigma_dual[p]))
    if t_prime_pr < 0:
        tpr_prime[p,r] = t_prime_pr

new_pairs = list(tpr_prime.keys())

# Add the new columns to the model
new_p = []
for p,r in new_pairs:
    new_p.append(p)

In [72]:
m2 = gp.Model('2nd PMF')

# Define the decision variables
# t_pr: number of passengers that would like to fly on itinerary p and are reallocated to itinerary r
t = {}
for p in path_list:
    for r in initial:
        t[p,r] = m2.addVar(name='t_'+str(p)+'_'+str(r))

# Add the new columns to the RMP
for p,r in new_pairs:
    t[p,r] = m2.addVar(name='t_' + str(p) +'_'+ str(r), lb =0)
m2.update()

# Update the objective function
of  = gp.quicksum((paths[p]['Fare'] - P_p[(p,r)]['Recapture rate'] * paths[r]['Fare']) * t[p,r] for r in initial for p in path_list)
of += gp.quicksum((paths[p]['Fare'] - P_p[(p,r)]['Recapture rate'] * paths[r]['Fare']) * t[p,r] for p,r in new_pairs)
m2.setObjective(of, GRB.MINIMIZE)

# Update the constraints
m2.addConstrs((gp.quicksum(s_ip[i, p] * t[p,r] for p in path_list for r in initial) +
              gp.quicksum(s_ip[i, p] * t[p,r] for p, r in new_pairs) -
              gp.quicksum(s_ip[i, r] * P_p[(p, r)]['Recapture rate'] * t[p,r] for p,r in new_pairs) >=
               ds_i[i] for i in flight_list), name='π')

# Constraint 2: sum t_pr <= Dp for all p
m2.addConstrs((t[p,r] <= paths[p]['Demand'] for p in path_list for r in initial), name='σ')
# Constraint 3: sum t_pr >= 0 for new pairs
m2.addConstrs((t[p,r] <= paths[p]['Demand'] for p, r in new_pairs), name='σ')

# Update the model
m2.update()

# Solve the model
m2.optimize()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 29 rows, 19 columns and 45 nonzeros
Model fingerprint: 0x6d6e5cde
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  Objective range  [5e+01, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+00, 2e+02]
Presolve removed 22 rows and 6 columns
Presolve time: 0.01s
Presolved: 7 rows, 13 columns, 19 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.1420000e+04   5.362500e+01   0.000000e+00      0s
       8    4.5006000e+04   0.000000e+00   0.000000e+00      0s

Solved in 8 iterations and 0.02 seconds (0.00 work units)
Optimal objective  4.500600000e+04


In [73]:
# Print the optimal objective value and the decision variables t_pr and the dual variables
print('Optimal objective value: %g' % m2.objVal)
print('Optimal solution:')
for v in m2.getVars():
    if v.x > 0:
        print('%s = %g' % (v.varName, v.x))

print('Dual variables:')
for c in m2.getConstrs():
    if c.Pi != 0:
        print('%s = %g' % (c.ConstrName, c.Pi))

# Save dual variables in a dictionary
pi_dual = {}
for c in m2.getConstrs():
    if c.constrName[0] == 'π':
        # get only the flight number from the constraint name
        flight_num_pi = int(re.findall(r'\d+', c.ConstrName)[0])    
        pi_dual[flight_num_pi] = c.Pi

sigma_dual = {}
for c in m2.getConstrs():
    if c.constrName[0] == 'σ':
        path_num_sigma = int(re.findall(r'\d+', c.ConstrName)[0])    
        sigma_dual[path_num_sigma] = c.Pi
        

Optimal objective value: 45006
Optimal solution:
t_5_999 = 50
t_6_999 = 12
t_7_999 = 63.2
t_8_999 = 70
t_9_999 = 32
t_10_999 = 56
t_13_999 = 30
t_2_1 = 12
t_3_4 = 31
t_12_11 = 72
Dual variables:
π[102] = 100
π[301] = 14
π[104] = 165
π[202] = 66
π[302] = 140
π[203] = 165
π[101] = 120
σ[5,999] = -90
σ[8,999] = -230
σ[13,999] = -86


In [74]:
# Constraints test
for i in flight_list:
    print('Flight number: ', i)
    for p in path_list:
        for r in initial:
            if s_ip[i,p] == 1:
                print(s_ip[i,p] * t[p,r])
    print('end sum 1')    
    for p, r in new_pairs:
        if s_ip[i,p] == 1:
            print(s_ip[i, p] * t[p,r])
    print('end sum 2')
    for p, r in new_pairs:
        if s_ip[i, r] == 1:
            print(s_ip[i, r] * P_p[(p, r)]['Recapture rate'] * t[p,r])
    print('end sum 3')
    print(ds_i[i], '\n')

Flight number:  102
t_1_999
t_7_999
end sum 1
end sum 2
0.1 t_2_1
end sum 3
62 

Flight number:  301
t_10_999
t_14_999
end sum 1
end sum 2
end sum 3
56 

Flight number:  201
t_5_999
t_7_999
t_11_999
end sum 1
end sum 2
0.1 t_12_11
end sum 3
36 

Flight number:  104
t_2_999
t_8_999
end sum 1
t_2_1
end sum 2
end sum 3
82 

Flight number:  202
t_3_999
t_10_999
t_13_999
end sum 1
t_3_4
end sum 2
end sum 3
117 

Flight number:  302
t_5_999
t_9_999
end sum 1
end sum 2
end sum 3
82 

Flight number:  303
t_15_999
end sum 1
end sum 2
end sum 3
-64 

Flight number:  204
t_4_999
end sum 1
end sum 2
0.2 t_3_4
end sum 3
-8 

Flight number:  203
t_8_999
t_12_999
end sum 1
t_12_11
end sum 2
end sum 3
142 

Flight number:  101
t_6_999
t_13_999
end sum 1
end sum 2
end sum 3
42 



In [75]:
# Create the pricing problem (PP)

tpr_prime = {}
# tpr = (fare_p - sum (π_i) for i being each flight in path p) - bpr * (fare_r - sum (π_j) for j being each flight in path p)) - σ_p
for p,r in P_p.keys():
    t_prime_pr = ((paths[p]['Fare'] - sum(pi_dual[i] for i in paths[p]['Legs'])) -
                      (P_p[(p,r)]['Recapture rate']) *
                      (paths[r]['Fare'] - sum(pi_dual[j] for j in paths[r]['Legs'])) -
                      (sigma_dual[p]))
    if t_prime_pr < 0:
        tpr_prime[p,r] = t_prime_pr

new_pairs = list(tpr_prime.keys())
print(new_pairs)

# Add the new columns to the model
new_p = []
for p,r in new_pairs:
    new_p.append(p)

if len(new_p) == 0:
    print('Optimal solution found')

[]
Optimal solution found


In [76]:
# Objective function test
for r in initial:
    for p in path_list:
        print((paths[p]['Fare'] - P_p[(p,r)]['Recapture rate'] * paths[r]['Fare']) * t[p,r])

for p,r in new_pairs:
    print(paths[p]['Fare'] - P_p[(p, r)]['Recapture rate'] * paths[r]['Fare'] * t[p,r])


150.0 t_1_999
170.0 t_2_999
80.0 t_3_999
70.0 t_4_999
50.0 t_5_999
120.0 t_6_999
100.0 t_7_999
100.0 t_8_999
140.0 t_9_999
80.0 t_10_999
150.0 t_11_999
180.0 t_12_999
100.0 t_13_999
75.0 t_14_999
80.0 t_15_999
0.0 t_999_999
